# PREPROCESAMIENTO - TRAIN

El objetivo del presente notebook consiste en la eliminación de outliers con base en variables previamente seleccionadas mediante análisis del diccionario de datos y tipo de datos por variable, junto con la dumización de las variables categóricas.

## SET UP

In [1]:
!pip install findspark

import findspark
findspark.init()

## LIBRERIAS

In [2]:
# Cargar Pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql.functions import *

spark = SparkSession.builder.appName("Test_spark").master("local[*]").getOrCreate()

In [3]:
from pyspark.sql.functions import *
from pyspark.sql import Window
from pyspark.sql.types import *
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoder

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## CARGAR DATOS

In [5]:
schema = StructType([
    StructField('periodo',IntegerType(), True),
    StructField('id_cli',IntegerType(), True),
    StructField('fecha_nacimiento',IntegerType(), True),
    StructField('edad',DoubleType(), True),
    StructField('genero',IntegerType(), True),
    StructField('estado_civil',StringType(), True),
    StructField('nivel_academico',StringType(), True),
    StructField('profesion',StringType(), True),
    StructField('ocupacion',StringType(), True),
    StructField('tipo_vivienda',StringType(), True),
    StructField('ult_actual',IntegerType(), True),
    StructField('categoria',DoubleType(), True),
    StructField('codigo_ciiu',IntegerType(), True),
    StructField('ind_mora_vigente',StringType(), True),
    StructField('cartera_castigada',StringType(), True),
    StructField('ciudad_residencia',StringType(), True),
    StructField('departamento_residencia',StringType(), True),
    StructField('ciudad_laboral',StringType(), True),
    StructField('departamento_laboral',StringType(), True),
    StructField('rechazo_credito',StringType(), True),
    StructField('mora_max',DoubleType(), True),
    StructField('cant_moras_30_ult_12_meses',DoubleType(), True),
    StructField('cant_moras_60_ult_12_meses',DoubleType(), True),
    StructField('cant_moras_90_ult_12_meses',DoubleType(), True),
    StructField('cupo_total_tc',DoubleType(), True),
    StructField('tenencia_tc',StringType(), True),
    StructField('cuota_tc_bancolombia',DoubleType(), True),
    StructField('tiene_consumo',StringType(), True),
    StructField('tiene_crediagil',StringType(), True),
    StructField('nro_tot_cuentas',IntegerType(), True),
    StructField('ctas_activas',IntegerType(), True),
    StructField('tiene_ctas_activas',StringType(), True),
    StructField('ctas_embargadas',IntegerType(), True),
    StructField('tiene_ctas_embargadas',StringType(), True),
    StructField('pension_fopep',StringType(), True),
    StructField('cuota_cred_hipot',DoubleType(), True),
    StructField('tiene_cred_hipo_1',StringType(), True),
    StructField('tiene_cred_hipo_2',StringType(), True),
    StructField('mediana_nom3',DoubleType(), True),
    StructField('mediana_pen3',DoubleType(), True),
    StructField('ingreso_nompen',DoubleType(), True),
    StructField('cat_ingreso',StringType(), True),
    StructField('ingreso_final',DoubleType(), True),
    StructField('cant_mora_30_tdc_ult_3m_sf',DoubleType(), True),
    StructField('cant_mora_30_consum_ult_3m_sf',DoubleType(), True),
    StructField('cuota_de_vivienda',DoubleType(), True),
    StructField('cuota_de_consumo',DoubleType(), True),
    StructField('cuota_rotativos',DoubleType(), True),
    StructField('cuota_tarjeta_de_credito',DoubleType(), True),
    StructField('cuota_de_sector_solidario',DoubleType(), True),
    StructField('cuota_sector_real_comercio',DoubleType(), True),
    StructField('cupo_tc_mdo',DoubleType(), True),
    StructField('saldo_prom3_tdc_mdo',DoubleType(), True),
    StructField('cuota_tc_mdo',DoubleType(), True),
    StructField('saldo_no_rot_mdo',DoubleType(), True),
    StructField('cuota_libranza_sf',DoubleType(), True),
    StructField('cant_oblig_tot_sf',DoubleType(), True),
    StructField('cant_cast_ult_12m_sr',DoubleType(), True),
    StructField('ind',DoubleType(), True),
    StructField('rep_calif_cred',IntegerType(), True),
    StructField('pol_centr_ext',DoubleType(), True),
    StructField('convenio_lib',StringType(), True),
    StructField('ingreso_nomina',DoubleType(), True),
    StructField('ingreso_segurida_social',DoubleType(), True),
    StructField('gasto_familiar',DoubleType(), True)
])

In [6]:
df = spark.read.schema(schema).option("delimiter", ";").option("nullValue", "\\N").csv("../iteration_1/input/Dataton_train_semicolon.csv")

In [7]:
df.count()

20988748

## PREPROCESAMIENTO

### Resumen estadístico por variable para obtener la reglas a aplicar en los outliers

In [8]:
summary = pd.read_csv("../iteration_1/output/statistics/summary.csv")
summary = summary.rename(columns={'Unnamed: 0': 'summary'})
summary = summary.set_index("summary")

In [9]:
summary.head()

,count,mean,stddev,min,25%,50%,75%,max,IQR,upper,lower
summary,,,,,,,,,,,
mora_max,15017120,5.008723e+00,2.371134e+01,0.0,0.0,0.0,1.0,999.000,1.0,2.5,-1.5
cupo_total_tc,20988748,4.426551e+06,1.093624e+07,0.0,0.0,0.0,4700000.0,9999999.000,4700000.0,11750000.0,-7050000.0
cuota_tc_bancolombia,20988748,1.949875e+05,6.079431e+05,0.0,0.0,0.0,197200.0,999999.999,197200.0,493000.0,-295800.0
nro_tot_cuentas,20988520,1.077879e+00,4.400288e-01,0.0,1.0,1.0,1.0,9.000,0.0,1.0,1.0
ctas_activas,20988520,1.020634e+00,4.069990e-01,0.0,1.0,1.0,1.0,9.000,0.0,1.0,1.0


In [10]:
summary.index.tolist()

['mora_max',
 'cupo_total_tc',
 'cuota_tc_bancolombia',
 'nro_tot_cuentas',
 'ctas_activas',
 'mediana_nom3',
 'mediana_pen3',
 'ingreso_final',
 'cuota_de_vivienda',
 'cuota_de_consumo',
 'cuota_rotativos',
 'cuota_tarjeta_de_credito',
 'cuota_de_sector_solidario',
 'cuota_sector_real_comercio',
 'cupo_tc_mdo',
 'saldo_prom3_tdc_mdo',
 'cuota_tc_mdo',
 'saldo_no_rot_mdo',
 'cuota_libranza_sf',
 'cant_oblig_tot_sf',
 'cant_cast_ult_12m_sr',
 'ind',
 'gasto_familiar']

### Imputaciones

In [11]:
data_imputation = df

In [12]:
# Zero
var_zero = [
    'categoria',
    'mora_max',
    'cant_moras_30_ult_12_meses',
    'cant_moras_60_ult_12_meses',
    'cant_moras_90_ult_12_meses',
    'nro_tot_cuentas',
    'ctas_activas',
    'ctas_embargadas',
    'cuota_cred_hipot',
    'cant_mora_30_tdc_ult_3m_sf',
    'cant_mora_30_consum_ult_3m_sf',
    'cuota_de_vivienda',
    'cuota_de_consumo',
    'cuota_rotativos',
    'cuota_tarjeta_de_credito',
    'cuota_sector_real_comercio',
    'cupo_tc_mdo',
    'saldo_prom3_tdc_mdo',
    'cuota_tc_mdo',
    'saldo_no_rot_mdo',
    'cuota_libranza_sf',
    'cant_oblig_tot_sf',
    'cant_cast_ult_12m_sr',
    'pol_centr_ext',
    'ingreso_segurida_social',
    'ingreso_nompen',
    'ingreso_nomina'
]

In [13]:
data_imputation = data_imputation.fillna(0,subset=var_zero)

In [14]:
# Median
var_med =[
    'cupo_total_tc',
    'cuota_tc_bancolombia',
    'mediana_nom3',
    'mediana_pen3',
    'ingreso_final',
    'ind',
]

In [15]:
for var in var_med:
    med = summary[summary.index == var].iloc[0]["50%"].astype("float64")
    data_imputation = data_imputation.fillna(med,subset=var)

### Eliminación de outliers

In [16]:
data_outliers = data_imputation

In [17]:
variables = ['ind',
             'ingreso_final',
             'cupo_total_tc',
             'cuota_tc_bancolombia',
             'mediana_nom3',
             'cuota_de_consumo',
             'cuota_tarjeta_de_credito',
             'cupo_tc_mdo',
             'saldo_prom3_tdc_mdo',
             'cuota_tc_mdo',
             'saldo_no_rot_mdo']

In [18]:
# Getting summary of variables
# Initialize missing variables
error = []
# Get the path
for var in variables:
    try:
        # Getting parameters
        upper =summary[summary.index == var].iloc[0]["upper"].astype("float64")
        lower = summary[summary.index == var].iloc[0]["lower"].astype("float64")
        # Dropping outliers
        data_outliers = data_outliers.where((col(var)>lower) & (col(var)<upper))
        print("variable: " + str(var),",rows: " + str(data_outliers.count()))
    except:
        # Getting missing variables
        error.append(var)
        pass 

variable: ind ,rows: 18361490
variable: ingreso_final ,rows: 17991726
variable: cupo_total_tc ,rows: 16360120
variable: cuota_tc_bancolombia ,rows: 15851845
variable: mediana_nom3 ,rows: 14914775
variable: cuota_de_consumo ,rows: 13976470
variable: cuota_tarjeta_de_credito ,rows: 13641098
variable: cupo_tc_mdo ,rows: 12979634
variable: saldo_prom3_tdc_mdo ,rows: 12100609
variable: cuota_tc_mdo ,rows: 11526291
variable: saldo_no_rot_mdo ,rows: 10768162


In [19]:
print(error)

[]


In [20]:
# Dropping outliers of Edad
data_outliers = data_outliers.withColumn("edad",round(col("edad"),0))
data_outliers = data_outliers.where((col("edad")<99) & (col("edad")>=18))
data_outliers.count()

10763851

In [21]:
# Drop rows with values smaller than zero
vars_smaller_zero = [
                  'cuota_de_vivienda',
                  'cuota_de_consumo',
                  'ind']

In [22]:
data_non_negatives = data_outliers

In [23]:
error = []
for var in vars_smaller_zero:
    try:
        data_non_negatives = data_non_negatives.where(col(var)>= 0)
        print("variable: " + str(var),",rows: " + str(data_non_negatives.count()))
    except:
    # Getting missing variables
        error.append(var)
        pass

variable: cuota_de_vivienda ,rows: 10763848
variable: cuota_de_consumo ,rows: 10761905
variable: ind ,rows: 9670308


In [24]:
print(error)

[]


## Limpieza de variables

In [46]:
data_clean = data_non_negatives

In [47]:
# Estas son las que tienen X o \N, pero son flags, entonces hay que cambiar 
# X por 1
flags = ['tiene_consumo', 
         'tiene_crediagil', 
         'tiene_ctas_activas', 
         'tiene_ctas_embargadas', 
         'pension_fopep', 
         'tiene_cred_hipo_1', 
         'tiene_cred_hipo_2']

In [48]:
for var in flags:
    data_clean =  data_clean.withColumn(var, when(col(var) == "X", "1").otherwise(0))

In [49]:
# Estas son varibles con respuesta S, N
# ind_mora_vigente N, S, \N
# cartera_castigada N, S, \N
sn_vars = ['ind_mora_vigente', 'cartera_castigada']

In [50]:
for var in sn_vars:
    #print (flag)
    #data = df.withColumn(flag, when(col(flag) == 'X', 1))
    data_clean =  data_clean.withColumn(var, when(col(var) == "S", "1").otherwise(0))

In [51]:
data_clean =  data_clean.withColumn("tenencia_tc", when(col("tenencia_tc") == "SI", "1").otherwise(0))

In [52]:
var_drop = [
    'genero',
    'profesion',
    'ult_actual',
    'codigo_ciiu',
    'ciudad_residencia',
    'ciudad_laboral',
    'departamento_laboral',
    'convenio_lib']

In [53]:
# Drop 
data_clean = data_clean.drop(*var_drop)

In [54]:
# Estado civil
data_clean = data_clean.withColumn("estado_civil"
                                   ,when(col("estado_civil") == "VIUDO",4)
                                   .when(col("estado_civil") == "SOLTERO",1)
                                   .when(col("estado_civil") == "NO INFORMA",0)
                                   .when(col("estado_civil") == "OTRO",0)
                                   .when(col("estado_civil") == "CASADO",3)
                                   .when(col("estado_civil") == "DIVORCIADO",5)
                                   .when(col("estado_civil") == "UNION LIBRE",2)
                                   .otherwise(0)
                                  )

In [55]:
# Nivel academico
data_clean = data_clean.withColumn("nivel_academico"
                                   ,when(col("nivel_academico") == "SIN INFORMACION",4)
                                   .when(col("nivel_academico") == "BACHILLER",1)
                                   .when(col("nivel_academico") == "NO INFORMA",0)
                                   .when(col("nivel_academico") == "ESPECIALIZACION",0)
                                   .when(col("nivel_academico") == "NINGUNO",3)
                                   .when(col("nivel_academico") == "TECNOLOGO",5)
                                   .when(col("nivel_academico") == "UNIVERSITARIO",2)
                                   .when(col("nivel_academico") == "PRIMARIO",2)
                                   .otherwise(0)
                                  )


In [56]:
#Ocupación
data_clean = data_clean.withColumn("ocupacion"
                                   ,when(col('ocupacion')=='Empleado','Empleado')
                                   .when(col('ocupacion')=='Independiente','Independiente')
                                   .when(col('ocupacion')=='Estudiante','Estudiante')
                                   .when(col('ocupacion')=='Profesional Independiente','Independiente')
                                   .when(col('ocupacion')=='Jubilado','Pensionado')
                                   .when(col('ocupacion')=='Comerciante','Independiente')
                                   .when(col('ocupacion')=='Socio Empleado - Socio','Empleado')
                                   .when(col('ocupacion')=='Ama de Casa','Independiente')
                                   .when(col('ocupacion')=='Otro','Indefinida')
                                   .when(col('ocupacion')=='Rentista de Capital','Independiente')
                                   .when(col('ocupacion')=='Pensionado','Pensionado')
                                   .when(col('ocupacion')=='Desempleado sin Ingresos','Desempleado')
                                   .when(col('ocupacion')=='Sin Ocupacion Asignada','Indefinida')
                                   .when(col('ocupacion')=='Desempleado con Ingresos','Independiente')
                                   .when(col('ocupacion')=='Ganadero','Independiente')
                                   .when(col('ocupacion')=='Agricultor','Independiente')
                                   .otherwise("Indefinida")
                                  )

In [57]:
# Tipo de vivienda
data_clean = data_clean.withColumn("tipo_vivienda"
                                   ,when(col("tipo_vivienda") == "ALQUILADA",1)
                                   .when(col("tipo_vivienda") == "NO INFORMA",0)
                                   .when(col("tipo_vivienda") == "PROPIA",3)
                                   .when(col("tipo_vivienda") == "FAMILIAR",2)
                                   .otherwise(0)
                                  )

In [58]:
# Rechazo Crédito
data_clean = data_clean.withColumn('rechazo_credito',
                                   when(col('rechazo_credito')=='RECHAZO BANCO',1)
                                   .when(col('rechazo_credito')=='RECHAZO FACTORING',1)
                                   .when(col('rechazo_credito')=='RECHAZO SUFI',1)
                                   .when(col('rechazo_credito')=='RECHAZO BIZAGI',1)
                                   .otherwise(0)
                                  )

In [59]:
#  Categoria de ingreso
data_clean = data_clean.withColumn('cat_ingreso'
                                   ,when(col('cat_ingreso') == "NOM",1)
                                   .when(col('cat_ingreso') == "PEN",2)
                                   .when(col('cat_ingreso') == "NOM PEN",3)
                                   .otherwise(0)
                                  )

In [60]:
# rep_calif_cred
data_clean = data_clean.withColumn('rep_calif_cred'
                                   ,when(col('rep_calif_cred') == "A",1)
                                   .when(col('rep_calif_cred') == "B",2)
                                   .when(col('rep_calif_cred') == "C",3)
                                   .when(col('rep_calif_cred') == "D",4)
                                   .when(col('rep_calif_cred') == "E",5)
                                   .when(col('rep_calif_cred') == "F",6)
                                   .when(col('rep_calif_cred') == "G",7)
                                   .when(col('rep_calif_cred') == "H",8)
                                   .otherwise(0)
                                  )

In [61]:
# Departamento
data_clean = data_clean.withColumn('departamento_residencia'
                                   ,when(col('departamento_residencia')=='ANTIOQUIA                                         ', 'ANDINA')
                                   .when(col('departamento_residencia')=='VALLE                                             ' , 'PACIFICO')
                                   .when(col('departamento_residencia')=='RISARALDA                                         ', 'ANDINA')
                                   .when(col('departamento_residencia')=='SANTANDER                                         ', 'ANDINA')
                                   .when(col('departamento_residencia')=='BOGOTA D.C.                                       ', 'ANDINA')
                                   .when(col('departamento_residencia')=='ESTADO EXTERIOR                                   ' , 'EXTERIOR')
                                   .when(col('departamento_residencia')=='QUINDIO                                           ', 'ANDINA')
                                   .when(col('departamento_residencia')=='BOLIVAR                                           ', 'CARIBE')
                                   .when(col('departamento_residencia')=='CORDOBA                                           ', 'CARIBE')
                                   .when(col('departamento_residencia')=='NORTE DE SANTANDER                                ', 'ANDINA')
                                   .when(col('departamento_residencia')=='ATLANTICO                                         ', 'CARIBE')
                                   .when(col('departamento_residencia')=='MAGDALENA                                         ' , 'CARIBE')
                                   .when(col('departamento_residencia')=='CUNDINAMARCA                                      ', 'ANDINA')
                                   .when(col('departamento_residencia')=='TOLIMA                                            ', 'ANDINA')
                                   .when(col('departamento_residencia')=='SUCRE                                             ', 'CARIBE')
                                   .when(col('departamento_residencia')=='CAUCA                                             ' , 'PACIFICO')
                                   .when(col('departamento_residencia')=='META                                              ' , 'ORINOQUIA')
                                   .when(col('departamento_residencia')=='CESAR                                             ' , 'CARIBE')
                                   .when(col('departamento_residencia')=='NARINO                                            ' , 'PACIFICO')
                                   .when(col('departamento_residencia')=='NI', 'NI')
                                   .when(col('departamento_residencia')=='BOYACA                                            ', 'ANDINA')
                                   .when(col('departamento_residencia')=='CAQUETA                                           ', 'AMAZONIA')
                                   .when(col('departamento_residencia')=='CALDAS                                            ', 'ANDINA')
                                   .when(col('departamento_residencia')=='HUILA                                             ', 'ANDINA')
                                   .when(col('departamento_residencia')=='CASANARE                                          ', 'ORINOQUIA')
                                   .when(col('departamento_residencia')=='SAN ANDRES Y PROVID                               ' , 'CARIBE')
                                   .when(col('departamento_residencia')=='NI                                   ', 'NI')
                                   .when(col('departamento_residencia')=='LA GUAJIRA                                        ', 'CARIBE')
                                   .when(col('departamento_residencia')=='CHOCO                                             ', 'PACIFICO')
                                   .when(col('departamento_residencia')=='ARAUCA                                            ', 'ORINOQUIA')
                                   .when(col('departamento_residencia')=='PUTUMAYO                                          ', 'AMAZONIA')
                                   .when(col('departamento_residencia')=='GUANIA                                            ', 'AMAZONIA')
                                   .when(col('departamento_residencia')=='GUAVIARE                                          ', 'AMAZONIA')
                                   .when(col('departamento_residencia')=='VAUPES                                            ', 'AMAZONIA')
                                   .when(col('departamento_residencia')=='AMAZONAS                                          ', 'AMAZONIA')
                                   .when(col('departamento_residencia')=='VICHADA                                           ', 'ORINOQUIA')
                                   .when(col('departamento_residencia')=='MADRID                                            ', 'EXTERIOR')
                                   .when(col('departamento_residencia')=='ESTADO DE LA FLORIDA                              ', 'EXTERIOR')
                                  )


In [62]:
data_dummy = data_clean

In [63]:
# Ocupación
data_dummy = (data_dummy
              .withColumn("Empleado",when(col("ocupacion")=="Empleado",1).otherwise(0))
              .withColumn("Independiente",when(col("ocupacion")=="Independiente",1).otherwise(0))
              .withColumn("Estudiante",when(col("ocupacion")=="Estudiante",1).otherwise(0))
              .withColumn("Pensionado",when(col("ocupacion")=="Pensionado",1).otherwise(0))
              .withColumn("Indefinida",when(col("ocupacion")=="Indefinida",1).otherwise(0))
              .withColumn("Desempleado",when(col("ocupacion")=="Desempleado",1).otherwise(0))
              .drop("ocupacion")
             )

In [64]:
# Departamento
data_dummy = (data_dummy
              .withColumn('ANDINA',when(col('departamento_residencia')=="ANDINA",1).otherwise(0))
              .withColumn('PACIFICO',when(col('departamento_residencia')=="PACIFICO",1).otherwise(0))
              .withColumn('EXTERIOR',when(col('departamento_residencia')=="EXTERIOR",1).otherwise(0))
              .withColumn('CARIBE',when(col('departamento_residencia')=="CARIBE",1).otherwise(0))
              .withColumn('ORINOQUIA',when(col('departamento_residencia')=="ORINOQUIA",1).otherwise(0))
              .withColumn('NI',when(col('departamento_residencia')=="NI",1).otherwise(0))
              .withColumn('AMAZONIA',when(col('departamento_residencia')=="AMAZONIA",1).otherwise(0))
              .drop("departamento_residencia")
             )

In [65]:
data_preprocessed = data_dummy

In [66]:
data_preprocessed.count()

9670308

In [67]:
# Save as parquet file
data_preprocessed.write.parquet("output/preprocessing/preprocessing_data.parquet")